<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in an Db2 database on IBM Cloud instance
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa

In [2]:
%load_ext sql

In [6]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL

###%sql ibm_db_sa://ksk44088:GJTxOOB84bKmq9vQ@55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31929/bludb?authSource=admin&replicaSet=replset

import ibm_db
%load_ext sql
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "ksk44088"        # e.g. "abc12345"
dsn_pwd = "GJTxOOB84bKmq9vQ"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "31929"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )
    
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31929;PROTOCOL=TCPIP;UID=ksk44088;PWD=GJTxOOB84bKmq9vQ;SECURITY=SSL;
Connected to database:  BLUDB as user:  ksk44088 on host:  55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0600
DB_NAME:    BLUDB
DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0500
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [24]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
###%sql select TABSCHEMA, TABNAME, CREATE_TIME from PROGRESS_REPORT_CARD'

sqlQuery="select SCHOOL_ID, NAME_OF_SCHOOL from PROGRESS_REPORT_CARDS limit 3"
stmt = ibm_db.exec_immediate(conn,sqlQuery)
ibm_db.fetch_both(stmt)


{'SCHOOL_ID': 610038,
 0: 610038,
 'NAME_OF_SCHOOL': 'Abraham Lincoln Elementary School',
 1: 'Abraham Lincoln Elementary School'}

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [28]:
# type in your query to retrieve the number of columns in the SCHOOLS table

sqlQuery="select count(*) from SYSCAT.COLUMNS where TABNAME = 'PROGRESS_REPORT_CARDS'"

stmt = ibm_db.exec_immediate(conn,sqlQuery)
ibm_db.fetch_both(stmt)

{'1': 78, 0: 78}

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [23]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
import pandas
import ibm_db_dbi
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pconn = ibm_db_dbi.Connection(conn)
df = pandas.read_sql("select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME='PROGRESS_REPORT_CARDS'", pconn)
df

,COLNAME,TYPENAME,LENGTH
0,SCHOOL_ID,INTEGER,4
1,NAME_OF_SCHOOL,VARCHAR,64
2,"Elementary, Middle, or High School",VARCHAR,2
3,STREET_ADDRESS,VARCHAR,29
4,CITY,VARCHAR,7
...,...,...,...
73,GRADUATION_RATE__,VARCHAR,4
74,COLLEGE_ENROLLMENT_RATE__,VARCHAR,4
75,COLLEGE_ENROLLMENT,SMALLINT,2
76,GENERAL_SERVICES_ROUTE,SMALLINT,2


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [35]:
###sqlQuery="select count(*) from PROGRESS_REPORT_CARDS where Elementary,Middle,or_High_School = 'EM'"
sqlQuery="select count(*) from SCHOOL_CARDS where SCHOOL_TYPE = 'ES'"

stmt = ibm_db.exec_immediate(conn,sqlQuery)
ibm_db.fetch_both(stmt)

{'1': 462, 0: 462}

Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where school_type = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [37]:
df = pandas.read_sql("select MAX(SAFETY_SCORE) AS MAX_SAFETY_SCORE from SCHOOL_CARDS", pconn)
df

,MAX_SAFETY_SCORE
0,99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [38]:
df = pandas.read_sql("select NAME_OF_SCHOOL from SCHOOL_CARDS where SAFETY_SCORE = (select MAX(SAFETY_SCORE) from SCHOOL_CARDS)", pconn)
df

,NAME_OF_SCHOOL
0,Abraham Lincoln Elementary School
1,Alexander Graham Bell Elementary School
2,Annie Keller Elementary Gifted Magnet School
3,Augustus H Burley Elementary School
4,Edgar Allan Poe Elementary Classical School
5,Edgebrook Elementary School
6,Ellen Mitchell Elementary School
7,James E McDade Elementary Classical School
8,James G Blaine Elementary School
9,LaSalle Elementary Language Academy


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [41]:
df = pandas.read_sql("select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from SCHOOL_CARDS order by AVERAGE_STUDENT_ATTENDANCE desc limit 10", pconn)
df

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,Velma F Thomas Early Childhood Center,None
1,John Charles Haines Elementary School,98.40%
2,James Ward Elementary School,97.80%
3,Edgar Allan Poe Elementary Classical School,97.60%
4,Rachel Carson Elementary School,97.60%
5,Orozco Fine Arts & Sciences Elementary School,97.60%
6,Annie Keller Elementary Gifted Magnet School,97.50%
7,Andrew Jackson Elementary Language Academy,97.40%
8,Lenart Elementary Regional Gifted Center,97.40%
9,Disney II Magnet School,97.30%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [42]:
df = pandas.read_sql("select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from SCHOOL_CARDS order by AVERAGE_STUDENT_ATTENDANCE asc limit 5", pconn)
df

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,Richard T Crane Technical Preparatory High School,57.90%
1,Barbara Vick Early Childhood & Family Center,60.90%
2,Dyett High School,62.50%
3,Wendell Phillips Academy High School,63.00%
4,Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [43]:
df = pandas.read_sql("select NAME_OF_SCHOOL, REPLACE(AVERAGE_STUDENT_ATTENDANCE,'%','') from SCHOOL_CARDS order by AVERAGE_STUDENT_ATTENDANCE asc limit 5", pconn)
df

,NAME_OF_SCHOOL,2
0,Richard T Crane Technical Preparatory High School,57.90
1,Barbara Vick Early Childhood & Family Center,60.90
2,Dyett High School,62.50
3,Wendell Phillips Academy High School,63.00
4,Orr Academy High School,66.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [49]:
df = pandas.read_sql("select NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from SCHOOL_CARDS where DECIMAL(REPLACE(AVERAGE_STUDENT_ATTENDANCE,'%','')) < 70 order by AVERAGE_STUDENT_ATTENDANCE desc", pconn)
df

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE
0,Roberto Clemente Community Academy High School,69.60%
1,Chicago Vocational Career Academy High School,68.80%
2,Manley Career Academy High School,66.80%
3,Orr Academy High School,66.30%
4,Wendell Phillips Academy High School,63.00%
5,Dyett High School,62.50%
6,Barbara Vick Early Childhood & Family Center,60.90%
7,Richard T Crane Technical Preparatory High School,57.90%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [51]:
sqlQrerry = "select distinct(COMMUNITY_AREA_NAME), sum(COLLEGE_ENROLLMENT) as TOTLE_COLLEGE_ENROLLMENT from SCHOOL_CARDS group by COMMUNITY_AREA_NAME"
df = pandas.read_sql(sqlQrerry, pconn)
df

,COMMUNITY_AREA_NAME,TOTLE_COLLEGE_ENROLLMENT
0,ALBANY PARK,6864
1,ARCHER HEIGHTS,4823
2,ARMOUR SQUARE,1458
3,ASHBURN,6483
4,AUBURN GRESHAM,4175
...,...,...
72,WEST LAWN,4207
73,WEST PULLMAN,3240
74,WEST RIDGE,8197
75,WEST TOWN,9429


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [57]:
sqlQrerry = "select distinct(COMMUNITY_AREA_NAME), sum(COLLEGE_ENROLLMENT) as TOTLE_COLLEGE_ENROLLMENT from SCHOOL_CARDS group by COMMUNITY_AREA_NAME order by TOTLE_COLLEGE_ENROLLMENT ASC limit 5"
df = pandas.read_sql(sqlQrerry, pconn)
df

,COMMUNITY_AREA_NAME,TOTLE_COLLEGE_ENROLLMENT
0,OAKLAND,140
1,FULLER PARK,531
2,BURNSIDE,549
3,OHARE,786
4,LOOP,871


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [56]:
sqlQrerry = "select NAME_OF_SCHOOL, SAFETY_SCORE from SCHOOL_CARDS order by SAFETY_SCORE ASC limit 5"
df = pandas.read_sql(sqlQrerry, pconn)
df

,NAME_OF_SCHOOL,SAFETY_SCORE
0,Edmond Burke Elementary School,1
1,Luke O'Toole Elementary School,5
2,George W Tilton Elementary School,6
3,Foster Park Elementary School,11
4,Emil G Hirsch Metropolitan High School,13


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score
FROM schools
ORDER BY safety_score
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [59]:
sqlQrerry = "select HARDSHIP_INDEX from CHICAGO_SOCIOECONOMIC_DATA CD, SCHOOL_CARDS SC where CD.CA = SC.COMMUNITY_AREA_NUMBER and SC.COLLEGE_ENROLLMENT = 4368"

df = pandas.read_sql(sqlQrerry, pconn)
df

,HARDSHIP_INDEX
0,6


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [63]:
sqlQrerry = "select HARDSHIP_INDEX from CHICAGO_SOCIOECONOMIC_DATA CD, SCHOOL_CARDS SC where CD.CA = SC.COMMUNITY_AREA_NUMBER and SC.COLLEGE_ENROLLMENT = (select MAX(COLLEGE_ENROLLMENT) from SCHOOL_CARDS)"

df = pandas.read_sql(sqlQrerry, pconn)
df

,HARDSHIP_INDEX
0,6


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2021-07-09        | 2.4     | Malika            | Updated connection string                 |
| 2021-05-19        | 2.3     | Lakshmi Holla     | Updated question                          |
| 2021-04-20        | 2.2     | Malika            | Added the libraries                       |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
